In [1]:
from datetime import datetime, timedelta
from analyze_stock import *  
from scrape_url import *

# Call the function and store the result
day = '2024-01-09'

# Set the time frame
date = datetime.strptime(day, '%Y-%m-%d')
end_date = date + timedelta(days=7)
start_date = date - timedelta(days=14)

# Check if start_date is earlier than today, if not change start_date to today - 7 days
current_date = datetime.now()
start_date = current_date - timedelta(days=14) if start_date >= current_date else start_date

# First page
url = 'https://finance.yahoo.com/calendar/earnings/?day=' + day
print(url)

ticker_data_list = process_earnings_table(extract_table(url))
    
# Second page
url += '&offset=100&size=100'

ticker_data_list = process_earnings_table(extract_table(url), ticker_data_list)

# Pandas Dataframe
ticker_data_sorted = convert_to_dataframe(ticker_data_list)

filtered_tickers = []

# Iterate through the sorted list of tickers
for index, row in ticker_data_sorted.iterrows():
    ticker = row['Symbol']
    
    if ticker in filtered_tickers:
        continue

    # Use the function from utils.py to analyze stock options
    options_metrics = analyze_stock_options(ticker)

    if options_metrics['total_itm_calls'] == 0 and options_metrics['total_itm_puts'] == 0:
        continue

    # Print 
    print_options_data(ticker, options_metrics)
    
    # Filtered tickers
    filtered_tickers.append(ticker)
    

https://finance.yahoo.com/calendar/earnings/?day=2024-01-09
Options data for WDFC:
Average IV for Calls: 0.3769811101797004
Average IV for Puts: nan
Total Call Volume: 139.0
Total Call open interest: 1072
Total Call engagement: 1211.0
Total Put Volume: 364.0
Total Put open interest: 860.0
Total Put engagement: 1224.0
Number of ITM Call Options: 25
Number of ITM Put Options: 6
Options data for AYI:
Average IV for Calls: 0.36549969562530515
Average IV for Puts: 0.41518149121767517
Total Call Volume: 134.0
Total Call open interest: 557
Total Call engagement: 691.0
Total Put Volume: 195.0
Total Put open interest: 2148
Total Put engagement: 2343.0
Number of ITM Call Options: 27
Number of ITM Put Options: 1
Options data for SNX:
Average IV for Calls: 0.28984651132812495
Average IV for Puts: 0.3546587565368652
Total Call Volume: 459.0
Total Call open interest: 1788
Total Call engagement: 2247.0
Total Put Volume: 486.0
Total Put open interest: 1699
Total Put engagement: 2185.0
Number of ITM Ca